In [42]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2
import os
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image as keras_image
from keras.utils import np_utils

In [ ]:
emotion_data=pd.read_csv('fer2013.csv')

In [12]:
X_train = []
y_train = []
X_test = []
y_test = []
for index, row in emotion_data.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

y_train= np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)


In [13]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(48,48,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


In [14]:
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,batch_size=32,epochs=100,verbose=1,validation_data=(X_test, y_test))

Train on 6023 samples, validate on 2935 samples
Epoch 1/10
6023/6023 [==============================] - 996s 165ms/step - loss: 13.6781 - acc: 0.1486 - val_loss: 14.0697 - val_acc: 0.1271
Epoch 2/10
6023/6023 [==============================] - 959s 159ms/step - loss: 13.7899 - acc: 0.1444 - val_loss: 14.0697 - val_acc: 0.1271
Epoch 3/10
6023/6023 [==============================] - 1113s 185ms/step - loss: 13.7417 - acc: 0.1474 - val_loss: 14.0697 - val_acc: 0.1271
Epoch 4/10
6023/6023 [==============================] - 1141s 189ms/step - loss: 13.7391 - acc: 0.1476 - val_loss: 14.0697 - val_acc: 0.1271
Epoch 5/10
6023/6023 [==============================] - 953s 158ms/step - loss: 13.8060 - acc: 0.1435 - val_loss: 14.0697 - val_acc: 0.1271
Epoch 6/10
6023/6023 [==============================] - 867s 144ms/step - loss: 13.7471 - acc: 0.1471 - val_loss: 14.0697 - val_acc: 0.1271
Epoch 7/10
6023/6023 [==============================] - 870s 144ms/step - loss: 13.7364 - acc: 0.1478 - val_lo

In [15]:

loss_and_metrics = model.evaluate(X_test,y_test)
print(loss_and_metrics)


2935/2935 [==============================] - 90s 31ms/step
[14.069696845553397, 0.12708688246584426]


In [16]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    model.save_weights("model.h5")
print("Model is saved")

Model is saved


In [5]:
del(model)

## Start webcam & test

In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import sys, getopt

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.models import model_from_json
from keras.preprocessing import image as keras_image
from keras.utils import np_utils
from keras.models import load_model

import warnings
warnings.filterwarnings('ignore')
# Just disables the warning, doesn't enable AVX/FMA
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

model = model_from_json(open("./ready/fer.json", "r").read())
model.load_weights('./ready/fer.h5')
#model=load_model('model.h5')


# openCV provides this xml data file to detect front face
#face_haar_cascade = cv2.CascadeClassifier('C:\\MyFolder\\Upgrad_pre_notebook\\emotion_detection\\haarcascade_frontalface_default.xml')
face_haar_cascade = cv2.CascadeClassifier(cv2.samples.findFile('haarcascade_frontalface_default.xml'))
#face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [42]:
cap=cv2.VideoCapture(0)

while (True):
    _ret, frame=cap.read()
    vis = frame.copy() # visual as output image
        
    grayFrame= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #grayFrame = cv2.equalizeHist(grayFrame)
    faces_detected = face_haar_cascade.detectMultiScale(grayFrame, 1.3  , 10)
    
    blue_color = (255, 0, 0)# Blue color in rectangle boundry
    green_color= (0, 255, 0)
    thickness = 2
    font=1
    
    for (x,y,w,h) in faces_detected:
        # Draw a rectangle with blue line borders of thickness of 2 px 
        cv2.rectangle(vis,(x,y), (x+w,y+h), blue_color,thickness)
        roi_gray=grayFrame[y:y+w,x:x+h]
        roi_gray=cv2.resize(roi_gray,(48,48))

        image_pixels = keras_image.img_to_array(roi_gray)   # image from keras.preprocessing
        image_pixels = np.expand_dims(image_pixels, axis = 0)
        image_pixels /= 255
    
    predictions = model.predict(image_pixels)
    max_index = np.argmax(predictions[0])
    

    emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    emotion_prediction = emotion_detection[max_index]

    label_position=(int(x), int(y))
    label=emotion_prediction

    cv2.putText(vis, label, label_position,cv2.FONT_HERSHEY_SIMPLEX,font,green_color,thickness,cv2.LINE_AA)
    #cv2.putText(image,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)  
    
    resized_image = cv2.resize(vis, (1000, 700))

    cv2.imshow('Detecting Emotion',vis)
    if cv2.waitKey(15) == ord('q'): # press key q to exit
        break

print('Done')
cap.release()
cv2.destroyAllWindows()

Done


In [2]:
from __future__ import division
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
from time import sleep
import os
import numpy as np
import cv2

#loading the model
json_file = open('./ready/fer.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./ready/fer.h5")
print("Loaded model from disk")

#setting image resizing parameters
WIDTH = 48
HEIGHT = 48
x=None
y=None
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
blue_color = (255, 0, 0)# Blue color in rectangle boundry
green_color= (0, 255, 0)
thickness = 2
font=1
face_haar_cascade = cv2.CascadeClassifier(cv2.samples.findFile('haarcascade_frontalface_default.xml'))

#loading image
cap=cv2.VideoCapture(0)

while (True):
    _ret, frame=cap.read()
    full_size_image = frame.copy() # visual as outp
    gray=cv2.cvtColor(full_size_image,cv2.COLOR_RGB2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray, 1.3  , 10)

    #detecting faces
    for (x, y, w, h) in faces_detected:
            roi_gray = gray[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
            cv2.rectangle(full_size_image, (x, y), (x + w, y + h), green_color, thickness)
            #predicting the emotion
            yhat= loaded_model.predict(cropped_img)
            cv2.putText(full_size_image, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, blue_color, thickness, cv2.LINE_AA)
            #print("Emotion: "+labels[int(np.argmax(yhat))])
            

    resized_image = cv2.resize(full_size_image, (1000, 700))

    cv2.imshow('Detecting Emotion',resized_image)
    sleep(0.30)
    if cv2.waitKey(30) == ord('q'): # press key q to exit
        break

print('Done')
cap.release()
cv2.destroyAllWindows()

Loaded model from disk
Done


In [ ]:
# Colour to grey image sample
import cv2
capture = cv2.VideoCapture(0)

while(True):
      
    ret, frame = capture.read()
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    cv2.imshow('video gray', grayFrame)
    cv2.imshow('video original', frame)
      
    if cv2.waitKey(1) == 27:  #ESC key press sends 27 
        break
        
capture.release()
cv2.destroyAllWindows()

In [1]:
#Sample from opencv github
import cv2 as cv

from video import create_capture
from common import clock, draw_str

def detect(img, cascade):
    rects = cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4, minSize=(30, 30),
                                     flags=cv.CASCADE_SCALE_IMAGE)
    if len(rects) == 0:
        return []
    rects[:,2:] += rects[:,:2]
    return rects

def draw_rects(img, rects, color):
    for x1, y1, x2, y2 in rects:
        cv.rectangle(img, (x1, y1), (x2, y2), color, 2)

def main():
    import sys, getopt
    video_src = 0
    ##nested_fn  = args.get('--nested-cascade', "data/haarcascades/haarcascade_eye.xml")
    ##'C:\\MyFolder\\Upgrad_pre_notebook\\emotion_detection\\haarcascade_frontalface_default.xml'
    cascade = cv.CascadeClassifier(cv.samples.findFile('haarcascade_frontalface_default.xml'))
    #nested = cv.CascadeClassifier(cv.samples.findFile(nested_fn))

    #cam = create_capture(video_src, fallback='synth:bg={}:noise=0.05'.format(cv.samples.findFile('lena.jpg')))
    cam = cv.VideoCapture(0)
    
    while True:
        _ret, img = cam.read()
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        gray = cv.equalizeHist(gray)

        t = clock()
        rects = detect(gray, cascade)
        vis = img.copy()
        draw_rects(vis, rects, (0, 255, 0))

        for x1, y1, x2, y2 in rects:
            roi = gray[y1:y2, x1:x2]
            vis_roi = vis[y1:y2, x1:x2]
            #subrects = detect(roi.copy(), nested)
            #draw_rects(vis_roi, subrects, (255, 0, 0))
        dt = clock() - t

        draw_str(vis, (20, 20), 'time: %.1f ms' % (dt*1000))
        cv.imshow('facedetect', vis)

        if cv.waitKey(5) == 27:d
            break

    print('Done')


if __name__ == '__main__':
    #print(__doc__)
    main()
    cv.destroyAllWindows()

NameError: name 'cv' is not defined